In [1]:
run Lattice_functions.ipynb

In [2]:
#Generating the lattice
nx = 50 + 1 # 1 added to make the total length = 100*0.01
ny = 50 + 1
lx = 0.02
ly = 0.02
k_inPlane = 10
nz = 3
lz = 0.01
k_vertical = 10
M1 = Rect_Lattice(nx, ny, lx, ly, k_inPlane, [], [], [])
M1 = add_layers_in_z_direction(M1, nz, lz, k_vertical, 0, [])

#setting z of the middle layer to zero:
min_x = M1.nodes[1].x
min_y = M1.nodes[1].y
max_x = M1.nodes[1].x
max_y = M1.nodes[1].y

for i in M1.nodes:
    i.z = round(i.z, 12) - lz
    i.x = round(i.x, 12)
    i.y = round(i.y, 12)
    min_x = min(i.x, min_x)
    min_y = min(i.y, min_y)
    max_x = max(i.x, max_x)
    max_y = max(i.y, max_y)

#Adding support
for i in M1.nodes:
    if(i.z == 0):
        if(i.x == min_x or i.x == max_x or i.y == min_y or i.y == max_y):
            i.s_x = 1
            i.s_y = 1
            i.s_z = 1
            M1.support_x.append(i.n)
            M1.support_y.append(i.n)
            M1.support_z.append(i.n)

In [3]:
#Optimization preparation:
x0 = []
y0 = []
z0 = []
for i in M1.nodes:
    x0.insert((i.n - 1), i.x)
    y0.insert((i.n - 1), i.y)
    z0.insert((i.n - 1), i.z)
    
coord0 = x0 + y0 + z0
nNodes = int(len(coord0)/3)

#bounds:
lb = -np.inf*np.ones(len(coord0))
ub = np.inf*np.ones(len(coord0))

for i in M1.support_x:
    lb[i - 1] = x0[i - 1]
    ub[i - 1] = x0[i - 1]

for i in M1.support_y:
    lb[nNodes + i - 1] = y0[i - 1]
    ub[nNodes + i - 1] = y0[i - 1]
    
for i in M1.support_z:
    lb[(2*nNodes) + i - 1] = z0[i - 1]
    ub[(2*nNodes) + i - 1] = z0[i - 1]


bounds = Bounds(lb, ub)

In [4]:
f = Point_load_at_centre(10, M1)
import time

In [ ]:
#trust-constr: Optimization Algorithm 1
start = time.time()
res1 = minimize(Energy, coord0, args = (f, M1), method='trust-constr', jac = Jacobian, options={'verbose': 1}, bounds=bounds)
end = time.time()

In [ ]:
result = res1.x
x_1 = result[0 : nNodes]
y_1 = result[nNodes : 2*nNodes]
z_1 = result[2*nNodes : 3*nNodes]

ax = Lattice_Plot(x_1, y_1, z_1, M1, [], 'DSZ', 30, 85, 0.5, 'result_from_trust_constr')

In [ ]:
print('Time taken by trust-constr: ' + str(end - start))
print('Optimized value of enery: ' + str(Energy(result, f, M1)))

In [ ]:
###########################################################################

In [ ]:
#SLSQP: Optimization Algorithm 2
res2 = minimize(Energy, coord0, args = (f, M1), method='SLSQP', jac = Jacobian, options={'disp': True}, bounds=bounds)

In [ ]:
result = res2.x
x_2 = result[0 : nNodes]
y_2 = result[nNodes : 2*nNodes]
z_2 = result[2*nNodes : 3*nNodes]

ax = Lattice_Plot(x_2, y_2, z_2, M1, [], 'XsY', 30, 85, 0.5, 'result_from_SLSQP')

In [ ]:
print('Time taken by SLSQP: ' + str(end - start))
print('Optimized value of enery: ' + str(Energy(result, f, M1)))

In [ ]:
###########################################################################

In [ ]:
#TNC: Optimization Algorithm 3
res3 = minimize(Energy, coord0, args = (f, M1), method='TNC', jac = Jacobian, options={'disp': True}, bounds=bounds)

In [ ]:
result = res3.x
x_3 = result[0 : nNodes]
y_3 = result[nNodes : 2*nNodes]
z_3 = result[2*nNodes : 3*nNodes]

ax = Lattice_Plot(x_3, y_3, z_3, M1, [], 'XsY', 30, 85, 0.5, 'result_from_TNC')

In [ ]:
print('Time taken by TNC: ' + str(end - start))
print('Optimized value of enery: ' + str(Energy(result, f, M1)))

In [ ]:
###########################################################################

In [ ]:
#L-BFGS-B: Optimization Algorithm 4
res4 = minimize(Energy, coord0, args = (f, M1), method='L-BFGS-B', jac = Jacobian, options={'iprint': 1}, bounds=bounds)

In [ ]:
result = res4.x
x_4 = result[0 : nNodes]
y_4 = result[nNodes : 2*nNodes]
z_4 = result[2*nNodes : 3*nNodes]

ax = Lattice_Plot(x_4, y_4, z_4, M1, [], 'XsY', 30, 85, 0.5, 'result_from_L_BFGS_B')

In [ ]:
print('Time taken by L-BFGS-B: ' + str(end - start))
print('Optimized value of enery: ' + str(Energy(result, f, M1)))